In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

In [2]:
train_df = pd.read_csv('2019loans.csv')
test_df = pd.read_csv('2020Q1loans.csv')


In [3]:
train_df

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,354912,354912,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,high_risk,n,...,100.0,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N
12176,354944,354944,15000.0,0.1774,540.34,RENT,50000.0,Verified,high_risk,n,...,90.5,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N
12177,354973,354973,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,high_risk,n,...,100.0,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N
12178,355002,355002,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,high_risk,n,...,100.0,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N


In [4]:
test_df['loan_status']

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
          ...    
4697    high_risk
4698    high_risk
4699    high_risk
4700    high_risk
4701    high_risk
Name: loan_status, Length: 4702, dtype: object

In [5]:

# print(train_df.columns.values)

In [6]:
train_df.shape

(12180, 86)

In [7]:
train_df['loan_status'].value_counts()

low_risk     6090
high_risk    6090
Name: loan_status, dtype: int64

Look for missing values (NaN)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12180 entries, 0 to 12179
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  12180 non-null  int64  
 1   index                       12180 non-null  int64  
 2   loan_amnt                   12180 non-null  float64
 3   int_rate                    12180 non-null  float64
 4   installment                 12180 non-null  float64
 5   home_ownership              12180 non-null  object 
 6   annual_inc                  12180 non-null  float64
 7   verification_status         12180 non-null  object 
 8   loan_status                 12180 non-null  object 
 9   pymnt_plan                  12180 non-null  object 
 10  dti                         12180 non-null  float64
 11  delinq_2yrs                 12180 non-null  float64
 12  inq_last_6mths              12180 non-null  float64
 13  open_acc                    121

In [9]:
train_df.isnull().sum()

Unnamed: 0                    0
index                         0
loan_amnt                     0
int_rate                      0
installment                   0
                             ..
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
hardship_flag                 0
debt_settlement_flag          0
Length: 86, dtype: int64

### Preprocessing
Convert categorical data into numerical

In [10]:
# # # Convert categorical data to numeric on training data, feature and target
# X_2019 = train_df.drop("loan_status", axis=1)
# X_2019 = pd.get_dummies(X_2019)
# train_df['loan_status'] = pd.get_dummies((train_df['loan_status']))
# from pandas import DataFrame
# y_2019 = DataFrame(train_df["loan_status"])

# # Convert categorical data to numeric on training data, feature and target
# X_2020test = test_df.drop("loan_status", axis=1)
# X_2020test = pd.get_dummies(X_2020test)

# test_df['loan_status'] = pd.get_dummies((test_df['loan_status']))
# from pandas import DataFrame
# y_2020test = DataFrame(test_df["loan_status"])
# print(X_2019.columns.values)

In [11]:
X_train=train_df.drop("loan_status", axis=1)
y_train=train_df['loan_status']
X_train=pd.get_dummies(X_train)

In [12]:
y_train.value_counts()

low_risk     6090
high_risk    6090
Name: loan_status, dtype: int64

In [13]:
X_train.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,57107,57107,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,...,0,1,0,1,1,0,1,0,1,0
1,141451,141451,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,...,0,1,0,1,1,0,1,0,1,0
2,321143,321143,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,...,0,1,0,1,1,0,1,0,1,0
3,11778,11778,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,...,0,1,0,1,1,0,1,0,1,0
4,169382,169382,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,...,0,1,0,1,1,0,1,0,1,0


In [14]:
y_train


0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
           ...    
12175    high_risk
12176    high_risk
12177    high_risk
12178    high_risk
12179    high_risk
Name: loan_status, Length: 12180, dtype: object

In [15]:
X_test=test_df.drop("loan_status", axis=1)
y_test=test_df['loan_status']
X_test=pd.get_dummies(X_test)

In [16]:
X_train.shape

(12180, 94)

In [17]:
X_test.shape

(4702, 93)

In [18]:
X_train.drop(['Unnamed: 0','index'],axis=1,inplace=True)
X_test.drop(['Unnamed: 0','index'],axis=1,inplace=True)

In [19]:
# Get missing columns in the training test
missing_cols = set( X_train.columns ) - set( X_test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test = X_test[X_train.columns]

In [20]:
X_test.shape

(4702, 92)

In [21]:
# Feature selection

In [22]:
y_test.value_counts()

low_risk     2351
high_risk    2351
Name: loan_status, dtype: int64

Remove features based on correlation

In [24]:
X_train.corr()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
loan_amnt,1.000000,-0.031855,0.931812,0.133745,0.040829,-0.029415,-0.038032,0.163135,-0.063349,0.307895,...,0.149074,NaN,-0.108467,0.108467,-0.147060,0.147060,-0.059159,0.059159,0.014014,-0.014014
int_rate,-0.031855,1.000000,0.046622,-0.041194,0.075444,0.102723,0.082870,-0.020202,0.012707,-0.046600,...,0.142173,NaN,0.039956,-0.039956,-0.069224,0.069224,-0.044594,0.044594,0.005293,-0.005293
installment,0.931812,0.046622,1.000000,0.131311,0.048119,-0.000554,-0.023644,0.153699,-0.056453,0.307908,...,0.138911,NaN,-0.079448,0.079448,-0.134692,0.134692,-0.064697,0.064697,0.012859,-0.012859
annual_inc,0.133745,-0.041194,0.131311,1.000000,-0.080211,0.012283,0.030294,0.071312,-0.023311,0.152846,...,-0.016818,NaN,-0.008891,0.008891,0.035419,-0.035419,-0.002319,0.002319,0.000228,-0.000228
dti,0.040829,0.075444,0.048119,-0.080211,1.000000,0.016811,-0.027007,0.078888,-0.009230,0.055366,...,0.100012,NaN,0.013953,-0.013953,-0.245272,0.245272,-0.006046,0.006046,-0.002410,0.002410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
application_type_Joint App,0.147060,0.069224,0.134692,-0.035419,0.245272,-0.009963,-0.038908,-0.034212,-0.015865,-0.014972,...,0.198338,NaN,-0.020418,0.020418,-1.000000,1.000000,-0.001594,0.001594,-0.003090,0.003090
hardship_flag_N,-0.059159,-0.044594,-0.064697,-0.002319,-0.006046,0.005786,-0.021692,-0.010991,0.005041,-0.008881,...,-0.006540,NaN,0.005688,-0.005688,0.001594,-0.001594,1.000000,-1.000000,-0.003475,0.003475
hardship_flag_Y,0.059159,0.044594,0.064697,0.002319,0.006046,-0.005786,0.021692,0.010991,-0.005041,0.008881,...,0.006540,NaN,-0.005688,0.005688,-0.001594,0.001594,-1.000000,1.000000,0.003475,-0.003475
debt_settlement_flag_N,0.014014,0.005293,0.012859,0.000228,-0.002410,0.006485,0.004142,0.001582,-0.005164,-0.008951,...,0.008977,NaN,0.006133,-0.006133,0.003090,-0.003090,-0.003475,0.003475,1.000000,-1.000000


In [25]:
cor_matrix = X_train.corr().abs()

In [26]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.90)]
print(to_drop)

['installment', 'out_prncp', 'out_prncp_inv', 'total_pymnt_inv', 'total_rec_prncp', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd', 'pub_rec_bankruptcies', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_il_high_credit_limit', 'initial_list_status_w', 'application_type_Joint App', 'hardship_flag_Y', 'debt_settlement_flag_Y']


In [27]:
len(to_drop)

16

In [28]:
X_train_drop=X_train.drop(to_drop,axis=1)
X_test_drop=X_test.drop(to_drop,axis=1)

Feature Scaling: Standarization, Normalization

In [29]:
# X_train, X_test, y_train, y_test = train_test_split(X_2019, y_2019, random_state=1)
# Counter(y_train)
scaler = StandardScaler().fit(X_train_drop)
X_train_scaled = scaler.transform(X_train_drop)
X_test_scaled = scaler.transform(X_test_drop)

## Testing Model 1 : Logistic Regression 

### Unscaled Data

In [30]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [31]:
# Fit (train) our model by using the training data
classifier.fit(X_train_drop, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=1000)

In [32]:
# Accuracy
# REcall
# Precision
# F1-Score
# R2 score

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
y_pred=classifier.predict(X_test_drop)

In [35]:
accuracy_score(y_test,y_pred)

0.6129306678009357

In [36]:
# Validate the model by using the test data
trainacc_log_unscaled=classifier.score(X_train_drop, y_train)
testacc_log_unscaled=classifier.score(X_test_drop, y_test)
print(f"Training Data Score: {classifier.score(X_train_drop, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_drop, y_test)}")

Training Data Score: 0.6665845648604269
Testing Data Score: 0.6129306678009357


In [37]:
# Split into testing and training data

In [38]:
from sklearn.metrics import classification_report,confusion_matrix

In [39]:
tn, fp, fn, tp=confusion_matrix(y_test,y_pred).ravel()

In [40]:
confusion_matrix(y_test,y_pred)

array([[1061, 1290],
       [ 530, 1821]])

In [41]:
tn,fp,fn,tp

(1061, 1290, 530, 1821)

In [42]:
print(classification_report(y_test, y_pred, target_names=classifier.classes_))

              precision    recall  f1-score   support

   high_risk       0.67      0.45      0.54      2351
    low_risk       0.59      0.77      0.67      2351

    accuracy                           0.61      4702
   macro avg       0.63      0.61      0.60      4702
weighted avg       0.63      0.61      0.60      4702



In [45]:
# get scores and classication report

In [46]:
## Rinse and Repeat

### Scaled Data

In [47]:
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [48]:
# Fit (train) our model by using the training data
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [49]:
y_pred_scaled=classifier.predict(X_test_scaled)

In [50]:
# Validate the model by using the test data
trainacc_log_scaled=classifier.score(X_train_scaled, y_train)
testacc_log_scaled=classifier.score(X_test_scaled, y_test)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.6859605911330049
Testing Data Score: 0.6816248404934071


In [51]:
confusion_matrix(y_test,y_pred_scaled)

array([[1500,  851],
       [ 646, 1705]])

In [52]:
print(classification_report(y_test, y_pred_scaled, target_names=classifier.classes_))

              precision    recall  f1-score   support

   high_risk       0.70      0.64      0.67      2351
    low_risk       0.67      0.73      0.69      2351

    accuracy                           0.68      4702
   macro avg       0.68      0.68      0.68      4702
weighted avg       0.68      0.68      0.68      4702



## Testing Model 2 : RandomForest

### Unscaled Data

In [53]:
# Train a Random Forest Classifier model on unscaled data and print the model score
# Fit random forest and get training and testing score
clf = RandomForestClassifier(random_state=1, n_estimators=500)
clf.fit(X_train_drop, y_train)

print(f'Training Score: {clf.score(X_train_drop, y_train)}')
print(f'Testing Score: {clf.score(X_test_drop, y_test)}')

Training Score: 1.0
Testing Score: 0.7103360272224585


In [54]:
trainacc_rf_unscaled=clf.score(X_train_drop, y_train)
testacc_rf_unscaled=clf.score(X_test_drop, y_test)

In [55]:
predictions = clf.predict(X_test_drop)
predictions

array(['low_risk', 'low_risk', 'low_risk', ..., 'high_risk', 'low_risk',
       'high_risk'], dtype=object)

In [56]:
confusion_matrix(y_test,predictions)

array([[1812,  539],
       [ 823, 1528]])

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=clf.classes_))

              precision    recall  f1-score   support

   high_risk       0.69      0.77      0.73      2351
    low_risk       0.74      0.65      0.69      2351

    accuracy                           0.71      4702
   macro avg       0.71      0.71      0.71      4702
weighted avg       0.71      0.71      0.71      4702



### Scaled Data

In [58]:
# scaler2=MinMaxScaler()
# scaler2.fit(X_train)
# X_train_norm=scaler2.transform(X_train)
# X_test_norm=scaler2.transform(X_test)

In [59]:
# Train a Random Forest Classifier model on the scaled data and print the model score
# Fit random forest and get training and testing score
clf = RandomForestClassifier(random_state=1,n_estimators=500)
clf.fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')


Training Score: 1.0
Testing Score: 0.7096980008507018


In [60]:
trainacc_rf_scaled=clf.score(X_train_scaled, y_train)
testacc_rf_scaled=clf.score(X_test_scaled, y_test)

In [61]:
predictions = clf.predict(X_test_scaled)
predictions

array(['low_risk', 'low_risk', 'low_risk', ..., 'high_risk', 'low_risk',
       'high_risk'], dtype=object)

In [62]:
confusion_matrix(y_test,predictions)

array([[1810,  541],
       [ 824, 1527]])

In [63]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["high_risk", "low_risk"]))

              precision    recall  f1-score   support

   high_risk       0.69      0.77      0.73      2351
    low_risk       0.74      0.65      0.69      2351

    accuracy                           0.71      4702
   macro avg       0.71      0.71      0.71      4702
weighted avg       0.71      0.71      0.71      4702



Assumptions before training:
- Random Forest will output better results (higher train and test accuracies) due to the model complexity, and due to the simplicity of Logistic Regression.
- Random Forest could potentially overfit the train data, which could lead to a bad performance on new data (test set)
- Due to the simplicity of Logictic Regression, the accuracies won't be good.

In [64]:
data=[['Logistic Regression Unscaled',trainacc_log_unscaled,testacc_log_unscaled],['Logistic Regression Scaled',trainacc_log_scaled,testacc_log_scaled],
      ['Random Forest Unscaled',trainacc_rf_unscaled,testacc_rf_unscaled],['Random Forest Scaled',trainacc_rf_scaled,testacc_rf_scaled]]

In [65]:
df=pd.DataFrame(data,columns=['Model','Train Accuracy','Test Accuracy'])

In [66]:
df

,Model,Train Accuracy,Test Accuracy
0,Logistic Regression Unscaled,0.666585,0.612931
1,Logistic Regression Scaled,0.685961,0.681625
2,Random Forest Unscaled,1.000000,0.710336
3,Random Forest Scaled,1.000000,0.709698
